Build a regression model.

In [17]:
import pandas as pd
import statsmodels.api as sm

df=pd.read_csv('../data/finaldf.csv')
df.head()
df=df.dropna(subset=['ratingyelp'])
df=df.dropna(subset=['ratingfoursq'])

Provide model output and an interpretation of the results. 

In [18]:
df['ratingyelp']=df[['ratingyelp']]*2
print(df[['distanceyelp', 'ratingyelp', 'free_bikes']].isnull().sum())
df['distanceyelp'] = pd.to_numeric(df['distanceyelp'], errors='coerce')
df['ratingyelp'] = pd.to_numeric(df['ratingyelp'], errors='coerce')
df['free_bikes'] = pd.to_numeric(df['free_bikes'], errors='coerce')
print(df.dtypes)



distanceyelp    0
ratingyelp      0
free_bikes      0
dtype: int64
station                    object
latitude                  float64
longitude                 float64
free_bikes                  int64
empty_slots                 int64
total_bike_cap              int64
distanceyelp              float64
poiname                    object
location.address           object
ratingyelp                float64
distancefoursq              int64
namefoursq                 object
location.addressfoursq     object
ratingfoursq              float64
popularityyelp            float64
categories                 object
dtype: object


In [19]:
X = df[['distanceyelp', 'ratingyelp']].dropna()
y = df['free_bikes'][X.index]  # Align y with the cleaned X
print(y.describe())


count    617.000000
mean       0.927066
std        0.968429
min        0.000000
25%        0.000000
50%        1.000000
75%        2.000000
max        4.000000
Name: free_bikes, dtype: float64


In [20]:
X = df[['distanceyelp', 'ratingyelp']]  # Characteristics of POIs
y = df['free_bikes'] 

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             free_bikes   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     4.120
Date:                Sat, 19 Oct 2024   Prob (F-statistic):             0.0167
Time:                        12:58:38   Log-Likelihood:                -851.08
No. Observations:                 617   AIC:                             1708.
Df Residuals:                     614   BIC:                             1721.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.1047      0.293      0.358   

# Stretch

How can you turn the regression model into a classification model?